# PRISM: Calculate A and B for Non-Precipitation

To use the AR, lag-1 approach for the non-precipitation parameters, two matrices are required to generate the solution for current values from lag-1 values and error terms.

In [1]:
%matplotlib inline

In [2]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
OUT_DIR = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Recharge\Data\JN' \
          r'otes\Processed\PRISM\Other_Weather'

In [4]:
IN_R0DP = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Rec' \
          r'harge\Data\JNotes\Processed\PRISM\Other_Weather\OWeath_Z_M0dp_1981-2010.pickle'
IN_R0RH = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Rec' \
          r'harge\Data\JNotes\Processed\PRISM\Other_Weather\OWeath_Z_M0rh_1981-2010.pickle'
IN_R1DP = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Rec' \
          r'harge\Data\JNotes\Processed\PRISM\Other_Weather\OWeath_Z_M1dp_1981-2010.pickle'
IN_R1RH = r'\\augustine.space.swri.edu\jdrive\Groundwater\R8937_Stochastic_CC_Rec' \
          r'harge\Data\JNotes\Processed\PRISM\Other_Weather\OWeath_Z_M1rh_1981-2010.pickle'

In [5]:
DPrho0DF = pd.read_pickle( IN_R0DP )

In [6]:
display( HTML( DPrho0DF.to_html() ))

,rho_X1,rho_X2,rho_X3
rho_1X,1.000000,0.526996,0.239470
rho_2X,0.526996,1.000000,0.679002
rho_3X,0.239470,0.679002,1.000000


In [7]:
RHrho0DF = pd.read_pickle( IN_R0RH )

In [8]:
display( HTML( RHrho0DF.to_html() ))

,rho_X1,rho_X2,rho_X3
rho_1X,1.000000,0.526996,-0.414894
rho_2X,0.526996,1.000000,0.138510
rho_3X,-0.414894,0.138510,1.000000


In [9]:
M0 = RHrho0DF.to_numpy()[:2,:2]

In [10]:
M0

array([[1.        , 0.52699567],
       [0.52699567, 1.        ]])

In [11]:
DPrho1DF = pd.read_pickle( IN_R1DP )

In [12]:
display( HTML( DPrho1DF.to_html() ))

,rho_X1_L1,rho_X2_L1,rho_X3_L1
rho_1X,0.662200,0.411151,0.140141
rho_2X,0.456128,0.720030,0.567805
rho_3X,0.196210,0.464581,0.693973


In [13]:
RHrho1DF = pd.read_pickle( IN_R1RH )

In [14]:
display( HTML( RHrho1DF.to_html() ))

,rho_X1_L1,rho_X2_L1,rho_X3_L1
rho_1X,0.662200,0.411151,-0.305127
rho_2X,0.456128,0.720030,0.156305
rho_3X,-0.260834,0.049142,0.598893


In [15]:
M1 = RHrho1DF.to_numpy()[:2,:2]

In [16]:
M1

array([[0.66219953, 0.41115142],
       [0.45612775, 0.72002951]])

## Matrix Inversion Solutions

\begin{equation*}
    M_{0} = \begin{bmatrix}
                1               & \rho_{0}(1,2)    & \rho_{0}(1,3) \\
                \rho_{0}(2,1)   &      1           & \rho_{0}(2,3) \\
                \rho_{0}(3,1)   & \rho_{0}(3,2)    & 1             \\
            \end{bmatrix}
\end{equation*}

\begin{equation*}
    M_{1} = \begin{bmatrix}
                \rho_{1}(1,1)   & \rho_{1}(1,2)    & \rho_{1}(1,3) \\
                \rho_{1}(2,1)   & \rho_{1}(2,2)    & \rho_{1}(2,3) \\
                \rho_{1}(3,1)   & \rho_{1}(3,2)    & \rho_{1}(3,3) \\
            \end{bmatrix}
\end{equation*}

\begin{equation*}
    A M_{0} = M_{1}
\end{equation*}

\begin{equation*}
    A = M_{0}^{-1} M_{1}
\end{equation*}

\begin{equation*}
    B B^{T} = M_{0} - A M_{1}^{T}
\end{equation*}

Then need to do Cholesky decomposition to get the lower-triangular matrix $B$ from $B B^{T}$

Testing part

In [48]:
M0List = [ [1.0, 0.633, 0.186 ], [0.633, 1.0, -0.193], [0.186, -0.193, 1.0 ] ]

In [49]:
M1List = [ [0.621, 0.445, 0.087 ], [0.563, 0.674, -0.100], [0.015, -0.091, 0.251 ] ]

In [50]:
M0L = np.array( M0List )
M1L = np.array( M1List )

In [51]:
M0L

array([[ 1.   ,  0.633,  0.186],
       [ 0.633,  1.   , -0.193],
       [ 0.186, -0.193,  1.   ]])

In [52]:
M1L

array([[ 0.621,  0.445,  0.087],
       [ 0.563,  0.674, -0.1  ],
       [ 0.015, -0.091,  0.251]])

In [53]:
M0Linv = np.linalg.inv( M0L )

In [54]:
M0Linv

array([[ 1.99732943, -1.3877001 , -0.63932939],
       [-1.3877001 ,  2.00283336,  0.64465906],
       [-0.63932939,  0.64465906,  1.24333447]])

In [55]:
ATest = np.matmul( M1L, M0Linv )

In [56]:
ATest

array([[ 0.56719337,  0.08558442, -0.00198017],
       [ 0.25311954,  0.50416862, -0.04977569],
       [-0.00423103, -0.04126391,  0.24382304]])

In [57]:
M1LTrans = M1L.T

In [58]:
M1LTrans

array([[ 0.621,  0.563,  0.015],
       [ 0.445,  0.674, -0.091],
       [ 0.087, -0.1  ,  0.251]])

In [59]:
M0L - np.matmul( np.matmul( M1L, M0Linv ), M1LTrans )

array([[ 0.60986012,  0.25578821,  0.18577731],
       [ 0.25578821,  0.51270648, -0.13842375],
       [ 0.18577731, -0.13842375,  0.93510887]])

In [60]:
STest = M0L - np.matmul( ATest, M1LTrans )

In [61]:
STest

array([[ 0.60986012,  0.25578821,  0.18577731],
       [ 0.25578821,  0.51270648, -0.13842375],
       [ 0.18577731, -0.13842375,  0.93510887]])

In [62]:
np.linalg.cholesky( STest )

array([[ 0.78093542,  0.        ,  0.        ],
       [ 0.3275408 ,  0.63672875,  0.        ],
       [ 0.23789074, -0.33977212,  0.87353979]])

In [65]:
TestA = np.matmul( M1L, np.linalg.inv( M0L ) )

In [66]:
TestA

array([[ 0.56719337,  0.08558442, -0.00198017],
       [ 0.25311954,  0.50416862, -0.04977569],
       [-0.00423103, -0.04126391,  0.24382304]])

In [67]:
TestB = np.linalg.cholesky( M0L - np.matmul( np.matmul( M1L, np.linalg.inv( M0L ) ), M1L.T ) )

In [68]:
TestB

array([[ 0.78093542,  0.        ,  0.        ],
       [ 0.3275408 ,  0.63672875,  0.        ],
       [ 0.23789074, -0.33977212,  0.87353979]])

Dew point calcs

In [69]:
M0DP = DPrho0DF.to_numpy()

In [70]:
M0DP

array([[1.        , 0.52699567, 0.23947026],
       [0.52699567, 1.        , 0.67900227],
       [0.23947026, 0.67900227, 1.        ]])

In [72]:
M1DP = DPrho1DF.to_numpy()

In [73]:
M1DP

array([[0.66219953, 0.41115142, 0.14014149],
       [0.45612775, 0.72002951, 0.56780522],
       [0.19620957, 0.46458097, 0.69397255]])

In [74]:
A_DP = np.matmul( M1DP, np.linalg.inv( M0DP ) )

In [75]:
A_DP

array([[ 0.59601076,  0.1833393 , -0.12707316],
       [ 0.13500771,  0.52934267,  0.17605001],
       [ 0.04955612, -0.04580331,  0.71320588]])

In [76]:
B_DP = np.linalg.cholesky( M0DP - np.matmul( np.matmul( M1DP, np.linalg.inv( M0DP ) ), M1DP.T ) )

In [77]:
B_DP

array([[0.74010132, 0.        , 0.        ],
       [0.26385809, 0.62265053, 0.        ],
       [0.16962081, 0.38490329, 0.58282836]])

Relative humidity 

In [78]:
M0RH = RHrho0DF.to_numpy()

In [79]:
M0RH

array([[ 1.        ,  0.52699567, -0.41489354],
       [ 0.52699567,  1.        ,  0.13851003],
       [-0.41489354,  0.13851003,  1.        ]])

In [80]:
M1RH = RHrho1DF.to_numpy()

In [81]:
M1RH

array([[ 0.66219953,  0.41115142, -0.30512675],
       [ 0.45612775,  0.72002951,  0.15630532],
       [-0.26083397,  0.04914214,  0.59889297]])

In [82]:
A_RH = np.matmul( M1RH, np.linalg.inv( M0RH ) )

In [83]:
A_RH

array([[ 0.5534305 ,  0.13249708, -0.09386418],
       [ 0.21855761,  0.58180281,  0.16639794],
       [ 0.016244  , -0.04415161,  0.61174794]])

In [84]:
B_RH = np.linalg.cholesky( M0RH - np.matmul( np.matmul( M1RH, np.linalg.inv( M0RH ) ), M1RH.T ) )

In [85]:
B_RH

array([[ 0.74189065,  0.        ,  0.        ],
       [ 0.26126535,  0.62219462,  0.        ],
       [-0.29766739,  0.23311334,  0.70504429]])

Just

In [86]:
M0 = M0RH[:2, :2].copy()
M1 = M1RH[:2, :2].copy()

In [87]:
M0

array([[1.        , 0.52699567],
       [0.52699567, 1.        ]])

In [88]:
M1

array([[0.66219953, 0.41115142],
       [0.45612775, 0.72002951]])

In [89]:
A = np.matmul( M1, np.linalg.inv( M0 ) )

In [90]:
A

array([[0.61683454, 0.0860823 ],
       [0.10615798, 0.66408471]])

In [91]:
B = np.linalg.cholesky( M0 - np.matmul( np.matmul( M1, np.linalg.inv( M0 ) ), M1.T ) )

In [92]:
B

array([[0.74574768, 0.        ],
       [0.24627435, 0.64246926]])